In [1]:
library(plyr, quietly = TRUE)
library(tidyverse, quietly = TRUE)
library(ggpubr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize() masks plyr::summarize()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘ggpubr’


The following ob

In [2]:
LLM_score_thresh = 0.8
enrichr_adj_pval_thresh = 0.05
enrichr_JI_thresh = 0.2

In [3]:
LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_DF.tsv", delim = "\t")

New names:
• `` -> `...1`
Rows: 11310 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (12): Source, GeneSetID, GeneSetName, GeneList, LLM Name, LLM Analysis, ...
dbl  (8): ...1, Unnamed: 0, n_Genes, Score, Rank, P-value, Adjusted P-value,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
dim(LLM_genes_DF)

[1] 11310    20

In [5]:
get_JI = function(GeneList, enrichr_genes){
    # assume separation is the same
    geneSetGenes = str_split(string = GeneList, pattern = " ")[[1]]
    enrichRGenes = str_split(string = enrichr_genes, pattern = " ")[[1]]
    JI = length(intersect(geneSetGenes, enrichRGenes))/ length(union(geneSetGenes, enrichRGenes))
    
    return(JI)
    }   

In [6]:
LLM_genes_DF = LLM_genes_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes))

In [ ]:
#LLM_genes_reduced_DF = LLM_genes_DF %>%
#group_by("Source", "GeneSetID", "GeneSetName")

In [7]:
## New 240613
reduced_LLM_genes_orig_DF = LLM_genes_DF %>%
group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
arrange(`Adjusted P-value`) %>%
filter(((n() > 1) & (enrichr_JI >= enrichr_JI_thresh) ) |
       (`Adjusted P-value` > enrichr_adj_pval_thresh) |
       (`Adjusted P-value` <= enrichr_adj_pval_thresh)) %>%
slice(1)

#filter() %>%
#arrange(`Adjusted P-value`) %>%
#slice(1)


In [8]:
dim(reduced_LLM_genes_orig_DF)

[1] 300  21

In [ ]:
## Original used in paper
# reduced_LLM_genes_orig_DF = LLM_genes_DF %>%
# group_by(Source, GeneSetID, GeneSetName, GeneList) %>%
# slice(which.max(enrichr_JI))

In [ ]:
dim(reduced_LLM_genes_orig_DF)

In [9]:
reduced_LLM_genes_DF = reduced_LLM_genes_orig_DF %>%
rowwise() %>%
mutate(enrichr_JI = get_JI(GeneList, GO_term_genes)) %>%
mutate(LLM_success_TF = ifelse(Score < 0.8, FALSE, TRUE),
       enrichr_success_TF = ifelse(((`Adjusted P-value` > enrichr_adj_pval_thresh) | (enrichr_JI < enrichr_JI_thresh)), FALSE, TRUE))

In [10]:
write_delim(x = reduced_LLM_genes_DF, file = "data/omics_revamped_LLM_Enrichr_simVals_failure_240613_firstJI_DF.tsv", delim = "\t")

In [ ]:
reduced_LLM_genes_DF = read_delim(file = "data/omics_revamped_LLM_Enrichr_simVals_failure_240613_firstJI_DF.tsv", delim = "\t")

In [11]:
reduced_LLM_genes_DF = reduced_LLM_genes_DF %>%
mutate(enrichr_adj_pVal = `Adjusted P-value`)

In [12]:
table(reduced_LLM_genes_DF[,c("LLM_success_TF", "Source")])

              Source
LLM_success_TF L1000 NeST Viral_Infections
         FALSE    80   46               41
         TRUE     46   80                7